# 효과적이고 효율적인 확산

[DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)을 사용하여 특정 스타일의 이미지를 생성하거나 원하는 것을 포함하는 것은 까다로울 수 있습니다. 종종 만족스러운 이미지를 얻기 전에 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)을 여러 번 실행해야 합니다. 그러나 아무것도 없는 상태에서 무언가를 생성하는 것은 계산 집약적인 프로세스이며, 특히 추론을 반복해서 실행하는 경우 더욱 그렇습니다.

이것이 파이프라인에서 최대한의 *계산* (속도) 및 *메모리* (GPU vRAM) 효율성을 얻어 추론 주기 사이의 시간을 줄여 더 빠르게 반복할 수 있도록 하는 것이 중요한 이유입니다.

이 튜토리얼에서는 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)을 사용하여 더 빠르고 더 나은 이미지를 생성하는 방법을 안내합니다.

먼저 [`runwayml/stable-diffusion-v1-5`](https://huggingface.co/runwayml/stable-diffusion-v1-5) 모델을 로드하여 시작합니다.

In [ ]:
from diffusers import DiffusionPipeline

model_id = "runwayml/stable-diffusion-v1-5"
pipeline = DiffusionPipeline.from_pretrained(model_id, use_safetensors=True)

사용할 예제 프롬프트는 늙은 전사 족장의 초상화이지만, 원하는 프롬프트를 자유롭게 사용하십시오.

In [ ]:
prompt = "늙은 전사 족장의 초상화 사진"

## 속도

<Tip>

💡 GPU에 액세스할 수 없는 경우 [Colab](https://colab.research.google.com/)과 같은 GPU 공급자로부터 무료로 사용할 수 있습니다!

</Tip>

추론 속도를 높이는 가장 간단한 방법 중 하나는 다른 PyTorch 모듈과 마찬가지로 파이프라인을 GPU에 배치하는 것입니다.

In [ ]:
pipeline = pipeline.to("cuda")

동일한 이미지를 사용하고 개선할 수 있도록 [`Generator`](https://pytorch.org/docs/stable/generated/torch.Generator.html)를 사용하고 [재현성](https://huggingface.co/docs/diffusers/main/en/./using-diffusers/reusing_seeds)을 위해 시드를 설정합니다.

In [ ]:
import torch

generator = torch.Generator("cuda").manual_seed(0)

이제 이미지를 생성할 수 있습니다.

In [ ]:
image = pipeline(prompt, generator=generator).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/stable_diffusion_101/sd_101_1.png">
</div>

이 프로세스는 T4 GPU에서 약 30초가 걸렸습니다(할당된 GPU가 T4보다 좋은 경우 더 빠를 수 있습니다). 기본적으로 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)은 50개의 추론 단계에 대해 전체 `float32` 정밀도로 추론을 실행합니다. `float16`과 같은 낮은 정밀도로 전환하거나 더 적은 추론 단계를 실행하여 이를 가속화할 수 있습니다.

`float16`으로 모델을 로드하고 이미지를 생성하여 시작하겠습니다.

In [ ]:
import torch

pipeline = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True)
pipeline = pipeline.to("cuda")
generator = torch.Generator("cuda").manual_seed(0)
image = pipeline(prompt, generator=generator).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/stable_diffusion_101/sd_101_2.png">
</div>

이번에는 이미지를 생성하는 데 약 11초밖에 걸리지 않았으며, 이는 이전보다 거의 3배 빠릅니다!

<Tip>

💡 파이프라인을 항상 `float16`으로 실행하는 것이 좋으며, 지금까지 출력 품질 저하를 거의 보지 못했습니다.

</Tip>

또 다른 옵션은 추론 단계 수를 줄이는 것입니다. 더 효율적인 스케줄러를 선택하면 출력 품질을 저하시키지 않고 단계 수를 줄이는 데 도움이 될 수 있습니다. [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)에서 `compatibles` 메서드를 호출하여 현재 모델과 호환되는 스케줄러를 찾을 수 있습니다.

In [ ]:
pipeline.scheduler.compatibles
[
    diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
    diffusers.schedulers.scheduling_unipc_multistep.UniPCMultistepScheduler,
    diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2DiscreteScheduler,
    diffusers.schedulers.scheduling_deis_multistep.DEISMultistepScheduler,
    diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
    diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
    diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
    diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
    diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2AncestralDiscreteScheduler,
    diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
    diffusers.schedulers.scheduling_heun_discrete.HeunDiscreteScheduler,
    diffusers.schedulers.scheduling_pndm.PNDMScheduler,
    diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
    diffusers.schedulers.scheduling_ddim.DDIMScheduler,
]

스테이블 디퓨전 모델은 기본적으로 [PNDMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/pndm#diffusers.PNDMScheduler)를 사용하며 일반적으로 약 50개의 추론 단계가 필요하지만 [DPMSolverMultistepScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/multistep_dpm_solver#diffusers.DPMSolverMultistepScheduler)와 같은 더 성능이 좋은 스케줄러는 약 20 또는 25개의 추론 단계만 필요합니다. [from_config()](https://huggingface.co/docs/diffusers/main/en/api/configuration#diffusers.ConfigMixin.from_config) 메서드를 사용하여 새 스케줄러를 로드합니다.

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

이제 `num_inference_steps`를 20으로 설정합니다.

In [ ]:
generator = torch.Generator("cuda").manual_seed(0)
image = pipeline(prompt, generator=generator, num_inference_steps=20).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/stable_diffusion_101/sd_101_3.png">
</div>

좋습니다. 추론 시간을 단 4초로 줄였습니다! ⚡️

## 메모리

파이프라인 성능을 향상시키는 또 다른 핵심은 메모리 소비를 줄이는 것입니다. 이는 간접적으로 더 빠른 속도를 의미합니다. 왜냐하면 종종 초당 생성되는 이미지 수를 최대화하려고 하기 때문입니다. 한 번에 얼마나 많은 이미지를 생성할 수 있는지 확인하는 가장 쉬운 방법은 `OutOfMemoryError` (OOM)가 발생할 때까지 다양한 배치 크기를 시도하는 것입니다.

프롬프트 및 `Generator` 목록에서 이미지 배치를 생성하는 함수를 만듭니다. 각 `Generator`에 시드를 할당하여 좋은 결과를 생성하면 재사용할 수 있도록 합니다.

In [ ]:
def get_inputs(batch_size=1):
    generator = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]
    prompts = batch_size * [prompt]
    num_inference_steps = 20

    return {"prompt": prompts, "generator": generator, "num_inference_steps": num_inference_steps}

`batch_size=4`로 시작하여 얼마나 많은 메모리를 소비했는지 확인합니다.

In [ ]:
from diffusers.utils import make_image_grid

images = pipeline(**get_inputs(batch_size=4)).images
make_image_grid(images, 2, 2)

더 많은 vRAM이 있는 GPU가 없는 한 위의 코드는 아마도 `OOM` 오류를 반환했을 것입니다! 대부분의 메모리는 교차 주의 계층에 의해 점유됩니다. 이 작업을 배치로 실행하는 대신 순차적으로 실행하여 상당한 양의 메모리를 절약할 수 있습니다. [enable_attention_slicing()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_attention_slicing) 함수를 사용하도록 파이프라인을 구성하기만 하면 됩니다.

In [ ]:
pipeline.enable_attention_slicing()

이제 `batch_size`를 8로 늘려보십시오!

In [ ]:
images = pipeline(**get_inputs(batch_size=8)).images
make_image_grid(images, rows=2, cols=4)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/stable_diffusion_101/sd_101_5.png">
</div>

이전에는 4개의 이미지 배치도 생성할 수 없었지만 이제는 이미지당 약 3.5초 만에 8개의 이미지 배치를 생성할 수 있습니다! 이것은 아마도 품질을 저하시키지 않고 T4 GPU에서 가장 빠르게 수행할 수 있는 속도일 것입니다.

## 품질

지난 두 섹션에서는 `fp16`을 사용하고, 더 성능이 좋은 스케줄러를 사용하여 추론 단계 수를 줄이고, 주의 슬라이싱을 활성화하여 메모리 소비를 줄임으로써 파이프라인 속도를 최적화하는 방법을 배웠습니다. 이제 생성된 이미지의 품질을 향상시키는 방법에 초점을 맞출 것입니다.

### 더 나은 체크포인트

가장 확실한 단계는 더 나은 체크포인트를 사용하는 것입니다. 스테이블 디퓨전 모델은 좋은 출발점이며, 공식 출시 이후 여러 개선된 버전도 출시되었습니다. 그러나 최신 버전을 사용한다고 해서 자동으로 더 나은 결과를 얻을 수 있는 것은 아닙니다. 여전히 다양한 체크포인트를 직접 실험하고 [부정적 프롬프트](https://minimaxir.com/2022/11/stable-diffusion-negative-prompt/) 사용과 같은 약간의 조사를 수행하여 최상의 결과를 얻어야 합니다.

분야가 성장함에 따라 특정 스타일을 생성하도록 미세 조정된 고품질 체크포인트가 점점 더 많아지고 있습니다. [허브](https://huggingface.co/models?library=diffusers&sort=downloads) 및 [Diffusers 갤러리](https://huggingface.co/spaces/huggingface-projects/diffusers-gallery)를 탐색하여 관심 있는 것을 찾아보십시오!

### 더 나은 파이프라인 구성 요소

현재 파이프라인 구성 요소를 최신 버전으로 교체해 볼 수도 있습니다. Stability AI의 최신 [자동 인코더](https://huggingface.co/stabilityai/stable-diffusion-2-1/tree/main/vae)를 파이프라인에 로드하고 일부 이미지를 생성해 보겠습니다.

In [ ]:
from diffusers import AutoencoderKL

vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
pipeline.vae = vae
images = pipeline(**get_inputs(batch_size=8)).images
make_image_grid(images, rows=2, cols=4)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/stable_diffusion_101/sd_101_6.png">
</div>

### 더 나은 프롬프트 엔지니어링

이미지를 생성하는 데 사용하는 텍스트 프롬프트는 매우 중요하며, 이를 *프롬프트 엔지니어링*이라고 합니다. 프롬프트 엔지니어링 중에 고려해야 할 몇 가지 사항은 다음과 같습니다.

- 생성하려는 이미지 또는 유사한 이미지가 인터넷에 어떻게 저장되어 있습니까?
- 원하는 스타일로 모델을 유도할 수 있는 추가 세부 정보는 무엇입니까?

이를 염두에 두고 색상 및 고품질 세부 정보를 포함하도록 프롬프트를 개선해 보겠습니다.

In [ ]:
prompt += ", 부족 표범 메이크업, 파란색 위에 빨간색, 측면 프로필, 시선 돌리기, 진지한 눈"
prompt += " 50mm 인물 사진, 강한 가장자리 조명 사진--beta --ar 2:3  --beta --upbeta"

새 프롬프트로 이미지 배치를 생성합니다.

In [ ]:
images = pipeline(**get_inputs(batch_size=8)).images
make_image_grid(images, rows=2, cols=4)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/stable_diffusion_101/sd_101_7.png">
</div>

꽤 인상적입니다! 시드가 `1`인 `Generator`에 해당하는 두 번째 이미지를 피사체의 나이에 대한 텍스트를 추가하여 조금 더 수정해 보겠습니다.

In [ ]:
prompts = [
    "가장 나이 많은 전사 족장의 초상화 사진, 부족 표범 메이크업, 파란색 위에 빨간색, 측면 프로필, 시선 돌리기, 진지한 눈 50mm 인물 사진, 강한 가장자리 조명 사진--beta --ar 2:3  --beta --upbeta",
    "늙은 전사 족장의 초상화 사진, 부족 표범 메이크업, 파란색 위에 빨간색, 측면 프로필, 시선 돌리기, 진지한 눈 50mm 인물 사진, 강한 가장자리 조명 사진--beta --ar 2:3  --beta --upbeta",
    "전사 족장의 초상화 사진, 부족 표범 메이크업, 파란색 위에 빨간색, 측면 프로필, 시선 돌리기, 진지한 눈 50mm 인물 사진, 강한 가장자리 조명 사진--beta --ar 2:3  --beta --upbeta",
    "젊은 전사 족장의 초상화 사진, 부족 표범 메이크업, 파란색 위에 빨간색, 측면 프로필, 시선 돌리기, 진지한 눈 50mm 인물 사진, 강한 가장자리 조명 사진--beta --ar 2:3  --beta --upbeta",
]

generator = [torch.Generator("cuda").manual_seed(1) for _ in range(len(prompts))]
images = pipeline(prompt=prompts, generator=generator, num_inference_steps=25).images
make_image_grid(images, 2, 2)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/stable_diffusion_101/sd_101_8.png">
</div>

## 다음 단계

이 튜토리얼에서는 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)을 계산 및 메모리 효율성에 맞게 최적화하고 생성된 출력의 품질을 향상시키는 방법을 배웠습니다. 파이프라인을 더욱 빠르게 만드는 데 관심이 있다면 다음 리소스를 살펴보십시오.

- [PyTorch 2.0](https://huggingface.co/docs/diffusers/main/en/./optimization/fp16) 및 [`torch.compile`](https://pytorch.org/docs/stable/generated/torch.compile.html)이 5~300% 더 빠른 추론 속도를 얻을 수 있는 방법을 알아보십시오. A100 GPU에서는 추론 속도가 최대 50%까지 빨라질 수 있습니다!
- PyTorch 2를 사용할 수 없는 경우 [xFormers](https://huggingface.co/docs/diffusers/main/en/./optimization/xformers)를 설치하는 것이 좋습니다. 메모리 효율적인 주의 메커니즘은 PyTorch 1.13.1과 잘 작동하여 속도를 높이고 메모리 소비를 줄입니다.
- 모델 오프로딩과 같은 기타 최적화 기술은 [이 가이드](https://huggingface.co/docs/diffusers/main/en/./optimization/fp16)에서 다룹니다.